In [2]:
import os
import sys
from pathlib import Path
PATH = os.path.join(os.getcwd(), '..', '..')
sys.path.append(PATH)

import torch
import numpy as np
import matplotlib.pyplot as plt

from fastai.vision import *

from src.model_utils.cyclegan import CycleGAN
from src.model_utils.databunch import ImageTupleList
from src.model_utils.callbacks import CycleGANTrainer
from src.model_utils.loss import CycleGanLoss

%load_ext autoreload
%autoreload 2

In [3]:
import os
from pathlib import Path
data_dir = Path('../../data/test')

In [4]:
data = (ImageTupleList.from_folders(data_dir, 'trainA', 'trainB')
                          .split_none()
                          .label_empty()
                          .transform(get_transforms(), size = 56)
                          .databunch(bs = 4))
cycle_gan = CycleGAN(ch_in = 3, ch_out = 3, 
                     disc_layers = 3, 
                     gen_blocks = 5)
learn = Learner(data, cycle_gan, 
                loss_func = CycleGanLoss(cycle_gan), 
                opt_func = partial(optim.Adam, betas = (0.5, 0.99)),
                callback_fns = [CycleGANTrainer])

In [7]:
learn.fit(2)

epoch,train_loss,valid_loss,id_loss,gen_loss,cyc_loss,D_A_loss,D_B_loss,time
0,8.137732,#na#,1.069611,4.928561,2.139559,1.355305,2.493279,02:00
1,14.759531,#na#,1.063506,4.913844,8.782182,1.353536,2.451282,03:14


> /userhome/34/h3509807/fastai/CycleGAN-MultiMNIST/src/model_utils/callbacks.py(77)on_epoch_end()
-> return add_metrics(last_metrics,
(Pdb) l
 72  	
 73  	    def on_epoch_end(self, last_metrics, **kwargs):
 74  	        # last_metrics is None
 75  	        # add_metrics is for updating last_metrics
 76  	        set_trace()
 77  ->	        return add_metrics(last_metrics,
 78  	                           [s.smooth for s in [
 79  	                               self.id_smter, self.gen_smter, self.cyc_smter, self.da_smter,self.db_smter
 80  	                           ]]
 81  	                          )
 82  	
(Pdb) last_metrics
[None]
(Pdb) from inspect import getsource
(Pdb) print(getsource(add_metrics))
def add_metrics(last_metrics:Collection[Rank0Tensor], mets:Union[Rank0Tensor, Collection[Rank0Tensor]]):
    "Return a dictionary for updating `last_metrics` with `mets`."
    last_metrics,mets = listify(last_metrics),listify(mets)
    return {'last_metrics': last_metrics + mets}

(